<a href="https://www.kaggle.com/code/abrosimovamaria/abrosimova-lr-14?scriptVersionId=129557415" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np 
from scipy.sparse import csr_matrix
!pip install pymorphy2
import pymorphy2
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report




In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/doc-cls/corp0.txt
/kaggle/input/output-files/y_data.txt
/kaggle/input/output-files/x_data.txt
/kaggle/input/new-data/f_y_dat.txt
/kaggle/input/new-data/f_x_dat.txt


In [5]:
file = open("../input/doc-cls/corp0.txt", "r")
filedata = file.readlines()
print("Размер dataset", len(filedata))

# dictionary = file.get_feature_names()
# print(" Размер словаря корпуса до предварительной обработки.", len(dictionary))


Размер dataset 3400


### 1. Подготовка набора данных.

Вариант токенизации:

·     ограничиваем максимальную длину документа (по числу словоформ);

·     приводим в нижний регистр;

·     иностранные слова заменяем на FRGN;

·     элементы с цифрами заменяем на NUMB;

·     удаляем знаки препинания;

·     Лемматизация русских слов. Её цель - сокращение словаря;

·     разделение данных и меток в разные файлы (x.txt, y.txt).

Лемматизация – это приведение словоформы к лемме, или нормальной форме. 

In [6]:
#Проверка. Пример предварительной обработки корпуса с выводом результатов для нескольких элементов датасета

dataset = filedata [35:39]#[54:60]
print("Dataset \n",dataset, "\n")
print("Размер dataset", len(dataset))

#делаем все слова нижнего регистра
a = [x.lower() for x in dataset]

# иностранные слова заменяем на FRGN; элементы с цифрами заменяем на NUMB
b = []
n = len(a)
for i in range (n):
    b.append(a[i].split(" "))
    for j in range (0, len(b[i])):
        if (re.search('[a-z]', b[i][j])):
            b[i][j]= "frgn"#"FRGN"
        if (re.search('[0-9]', b[i][j])):
            b[i][j]= "numb"#"NUMB"
    b[i]=" ".join(b[i])
    
#print(b)

out = []
for x in b:  
     out.append (re.sub(r'[^\w\s]','', x))

dataset = out
print("--------------------------------------")
print("dataset после подготовки данных")
print(dataset)

  



Dataset 
 ['#семья В реальности же чем больше у пары свободы в выборе различных ролей, тем она гибче. Основная задача для супругов - пересмотреть свои потребности, определить свои выгоды от занимаемых ролей и найти свое собственное творческое решение. И тогда вдруг оказывается, что мужу, например, удобно сидеть дома с детьми, а жене нравится зарабатывать и чувствовать себя успешной, и иногда им важно поиграть друг с другом в дочки-матери, выполняя роли родителя или ребенка. И пара чертовски счастлива, и наплевать им на общественные стереотипы!\n', '#семья Да, это прекрасно, когда партнеры могут много давать друг другу. Да, семья — это то место, где человек может получить то, что он не получит нигде. Но важно осознать, что даже встретив партнера своей мечты, ты не попадешь в рай, и большинство твоих личных «тараканов», нерешенных внутренних проблем и страхов не исчезнут как по мановению волшебной палочки. У каждого человека свой путь и своя ответственность за то, чтобы пройти его, преод

In [7]:
# лемматизация
# Заменяет  слова леммами 
def to_normal_form(morph, s): 
    s2 = s.split() # Список слов предложения s 
    s = '' 
    for w in s2: 
        w = morph.parse(w)[0].normal_form 
        s += (' ' + w) 
    return s.lstrip() 
morph = pymorphy2.MorphAnalyzer() 
for i in range (len(dataset)):
    dataset[i]=to_normal_form(morph, dataset[i])
print (dataset)
    

['семья в реальность же чем большой у пара свобода в выбор различный роль тем она гибкий основный задача для супруг пересмотреть свой потребность определить свой выгода от занимать роль и найти свой собственный творческий решение и тогда вдруг оказываться что муж например удобно сидеть дом с ребёнок а жена нравиться зарабатывать и чувствовать себя успешный и иногда они важно поиграть друг с друг в дочкиматерить выполнять роль родитель или ребёнок и пара чертовски счастливый и наплевать они на общественный стереотип', 'семья да это прекрасно когда партнёр мочь много давать друг друг да семья это то место где человек мочь получить то что он не получить нигде но важно осознать что даже встретить партнёр свой мечта ты не попасть в рай и большинство твой личный таракан нерешённый внутренний проблема и страх не исчезнуть как по мановение волшебный палочка у каждый человек свой путь и свой ответственность за то чтобы пройти он преодолевать внутренний сопротивление хороший партнёр значительно 

In [9]:
dataset = out

#извлекаем метки
print("--------------------------------------")
y_data = []
n = len (dataset)
for i in range (n):
    y_words = dataset[i].split(" ")
    y_data.append(y_words[0])
    
#print("Разбиение предложения по словам y_words", y_words,"\n")
print("Метки y_label, \n", y_data,"\n")
#извлекаем предложения без меток
print("--------------------------------------")
x_data = []
n = len (dataset)
for i in range (n):
    ind_begin = dataset[i].index(" ")
    ind_begin +=1
    x_data.append(dataset[i][ind_begin:])
    
print("Предложения без меток: \n", x_data,"\n")

f_x = open("f_x.txt", "w")
f_y = open("f_y.txt", "w")

np.savetxt(f_x, x_data, fmt='%s')
np.savetxt(f_y, y_data, fmt='%s')

f_x.close()
f_y.close() 


# x_data = np.array(x_data)
# y_data = np.array(y_data)

# for  line in x_data:
#      f_x.write(line+"\n")
# for  line in y_data:
#      f_y.write(line+"\n")
    

--------------------------------------
Метки y_label, 
 ['семья', 'семья', 'реклама', 'реклама'] 

--------------------------------------
Предложения без меток: 
 ['в реальность же чем большой у пара свобода в выбор различный роль тем она гибкий основный задача для супруг пересмотреть свой потребность определить свой выгода от занимать роль и найти свой собственный творческий решение и тогда вдруг оказываться что муж например удобно сидеть дом с ребёнок а жена нравиться зарабатывать и чувствовать себя успешный и иногда они важно поиграть друг с друг в дочкиматерить выполнять роль родитель или ребёнок и пара чертовски счастливый и наплевать они на общественный стереотип', 'да это прекрасно когда партнёр мочь много давать друг друг да семья это то место где человек мочь получить то что он не получить нигде но важно осознать что даже встретить партнёр свой мечта ты не попасть в рай и большинство твой личный таракан нерешённый внутренний проблема и страх не исчезнуть как по мановение волше

# Основная программа

### Проверка. Пример предварительной обработки корпуса с выводом результатов для нескольких элементов датасета

In [10]:
# #Проверка. Пример предварительной обработки корпуса с выводом результатов для нескольких элементов датасета
# dataset = filedata 
# #print("Dataset \n",dataset, "\n")
# print("Размер dataset", len(dataset))

# #делаем все слова нижнего регистра
# a = [x.lower() for x in dataset]

# # иностранные слова заменяем на FRGN; элементы с цифрами заменяем на NUMB
# b = []
# n = len(a)
# for i in range (n):
#     b.append(a[i].split(" "))
#     for j in range (0, len(b[i])):
#         if (re.search('[a-z]', b[i][j])):
#             b[i][j]= "frgn"#"FRGN"
#         if (re.search('[0-9]', b[i][j])):
#             b[i][j]= "numb"#"NUMB"
#     b[i]=" ".join(b[i])
    
# #print(b)
# out = []
# for x in b:  
#      out.append (re.sub(r'[^\w\s]','', x))

# dataset = out
# # print("--------------------------------------")
# # print("dataset после подготовки данных")
# # print(dataset)
# print("Результат получен")

  



###  лемматизация

In [11]:
# # лемматизация
# # Заменяет  слова леммами 
# def to_normal_form(morph, s): 
#     s2 = s.split() # Список слов предложения s 
#     s = '' 
#     for w in s2: 
#         w = morph.parse(w)[0].normal_form 
#         s += (' ' + w) 
#     return s.lstrip() 
# morph = pymorphy2.MorphAnalyzer() 
# for i in range (len(dataset)):
#     dataset[i]=to_normal_form(morph, dataset[i])
# #print (dataset)
# print("Результат получен")
    

In [12]:
# dataset = out

# #извлекаем метки
# y_data = []
# n = len (dataset)
# for i in range (n):
#     y_words = dataset[i].split(" ")
#     y_data.append(y_words[0])
    
# #print("Разбиение предложения по словам y_words", y_words,"\n")
# #print("Метки y_label, \n", y_data,"\n")
# #извлекаем предложения без меток
# x_data = []
# n = len (dataset)
# for i in range (n):
#     ind_begin = dataset[i].index(" ")
#     #ind_end = dataset[i].index("\n")
#     ind_begin +=1
#     #x_data.append(dataset[i][ind_begin:ind_end])
#     x_data.append(dataset[i][ind_begin:])
    
# #print("Предложения без меток: \n", x_data,"\n")

# f_x = open("f_x.txt", "w")
# f_y = open("f_y.txt", "w")

# np.savetxt(f_x, x_data, fmt='%s')
# np.savetxt(f_y, y_data, fmt='%s')

# f_x.close()
# f_y.close() 
# print("Данные записаны в файлы")
    

### 2. Делим набор данных на обучающее  и проверочное множества.

In [13]:
x_file = open("../input/new-data/f_x_dat.txt", "r")
x_data = x_file.readlines()
print("Размер dataset", len(x_data))

y_file = open("../input/new-data/f_y_dat.txt", "r")
y_data = y_file.readlines()
print("Кол-во меток", len(y_data))


#<загрузка  данных и меток в списки (массивы) x, y>
k_split = 0.2

x = x_data
y = y_data

from sklearn.preprocessing import LabelEncoder
import keras
lencoder = LabelEncoder()
lencoder.fit(y_data)

x_trn, x_vl, y_trn, y_vl = train_test_split(x, y, test_size = k_split, shuffle = True)
print( "кол-во данных в x_trn", len(x_trn))
print( "кол-во данных в x_ vl", len(x_vl))
#print(x_trn,"\n", x_vl,"\n", y_trn,"\n", y_vl)



Размер dataset 3400
Кол-во меток 3400
кол-во данных в x_trn 2720
кол-во данных в x_ vl 680


### 3. Векторизация.

In [14]:
from sklearn.feature_extraction.text import CountVectorizer 
vec = CountVectorizer(token_pattern = '\w+', binary = False) # Если задать ,binary = True то векторы, представляющие строки текста, будут бинарными
len_trn = len(x_trn) 
x_trn.extend(x_vl) # Объединяем x_trn и x_vl и получаем полный корпус 
x_trn = vec.fit_transform(x_trn) 
x_vl = x_trn[len_trn:] 
x_trn = x_trn[:len_trn] 
# Преобразуем разреженные матрицы в массивы 
x_trn = np.float32(x_trn.toarray()) 
x_vl = np.float32(x_vl.toarray())
print(x_trn.shape)
print(x_vl.shape)

print(len(vec.get_feature_names()))

(2720, 34707)
(680, 34707)
34707


In [15]:
import time

### SGDClassifier.

In [16]:
from sklearn.linear_model import SGDClassifier 
doc_clf = SGDClassifier(loss = 'hinge', max_iter = 1000, tol = 1e-3) 

start_time = time.time()
#print(y_trn)
doc_clf.fit(x_trn, y_trn) # Обучение классификатора 
end_time = time.time() - start_time
print("Время выполнения :  {:.8f} секунд " .format(end_time) )
print('Оценка точности классификации') 
score = doc_clf.score(x_vl, y_vl) 
print('Точность на проверочном множестве:', round(score, 4)) 
score = doc_clf.score(x_trn, y_trn) 
print('Точность на обучающем множестве:', round(score, 4))

predictions = doc_clf.predict(x_vl)
print(classification_report(lencoder.transform(y_vl), lencoder.transform(predictions), digits = 4, target_names = lencoder.classes_))



Время выполнения :  33.45445490 секунд 
Оценка точности классификации
Точность на проверочном множестве: 0.8074
Точность на обучающем множестве: 0.9688
               precision    recall  f1-score   support

  автомобиль
     0.9487    0.7872    0.8605        47
    здоровье
     0.4058    0.8235    0.5437        34
    культура
     0.7867    0.8194    0.8027        72
       наука
     0.9167    0.9565    0.9362        46
недвижимость
     1.0000    0.1579    0.2727        19
    политика
     0.9151    0.8818    0.8981       110
происшествие
     0.8723    0.9213    0.8962        89
     реклама
     0.7000    0.2800    0.4000        25
       семья
     0.7857    0.5000    0.6111        22
       спорт
     0.8780    0.9863    0.9290        73
      страна
     0.7647    0.5417    0.6341        24
     техника
     0.7013    0.8852    0.7826        61
   экономика
     0.9130    0.7241    0.8077        58

     accuracy                         0.8074       680
    macro avg     0.8

### LogisticRegression

In [17]:
from sklearn.linear_model import LogisticRegression 
doc_clf = LogisticRegression(solver = 'lbfgs', # newton-cg 
                             max_iter = 500, multi_class = 'auto') 
start_time = time.time()
doc_clf.fit(x_trn, y_trn) # Обучение классификатора 
end_time = time.time() - start_time
print("Время выполнения :  {:.8f} секунд " .format(end_time) )
print('Оценка точности классификации') 
score = doc_clf.score(x_vl, y_vl) 
print('Точность на проверочном множестве:', round(score, 4)) 
score = doc_clf.score(x_trn, y_trn) 
print('Точность на обучающем множестве:', round(score, 4))

predictions = doc_clf.predict(x_vl)
print(classification_report(lencoder.transform(y_vl), lencoder.transform(predictions), digits = 4, target_names = lencoder.classes_))


Время выполнения :  150.60388708 секунд 
Оценка точности классификации
Точность на проверочном множестве: 0.8309
Точность на обучающем множестве: 1.0
               precision    recall  f1-score   support

  автомобиль
     0.8478    0.8298    0.8387        47
    здоровье
     0.6170    0.8529    0.7160        34
    культура
     0.7778    0.7778    0.7778        72
       наука
     0.9545    0.9130    0.9333        46
недвижимость
     0.6500    0.6842    0.6667        19
    политика
     0.9057    0.8727    0.8889       110
происшествие
     0.9000    0.9101    0.9050        89
     реклама
     0.8333    0.4000    0.5405        25
       семья
     0.7273    0.7273    0.7273        22
       спорт
     0.9333    0.9589    0.9459        73
      страна
     0.5789    0.4583    0.5116        24
     техника
     0.7361    0.8689    0.7970        61
   экономика
     0.8909    0.8448    0.8673        58

     accuracy                         0.8309       680
    macro avg     0.796

In [18]:
#---------------------
from sklearn.svm import SVC 
doc_clf_svc = SVC()
#doc_clf = SGDClassifier(loss = 'hinge', max_iter = 1000, tol = 1e-3) 

start_time = time.time()
#print(y_trn)
doc_clf_svc.fit(x_trn, y_trn) # Обучение классификатора 
end_time = time.time() - start_time
print("Время выполнения :  {:.8f} секунд " .format(end_time) )
print('Оценка точности классификации') 
score = doc_clf_svc.score(x_vl, y_vl) 
print('Точность на проверочном множестве:', round(score, 4)) 
score = doc_clf_svc.score(x_trn, y_trn) 
print('Точность на обучающем множестве:', round(score, 4))

predictions = doc_clf.predict(x_vl)
print(classification_report(lencoder.transform(y_vl), lencoder.transform(predictions), digits = 4, target_names = lencoder.classes_))

Время выполнения :  453.30653548 секунд 
Оценка точности классификации
Точность на проверочном множестве: 0.6838
Точность на обучающем множестве: 0.8169
               precision    recall  f1-score   support

  автомобиль
     0.8478    0.8298    0.8387        47
    здоровье
     0.6170    0.8529    0.7160        34
    культура
     0.7778    0.7778    0.7778        72
       наука
     0.9545    0.9130    0.9333        46
недвижимость
     0.6500    0.6842    0.6667        19
    политика
     0.9057    0.8727    0.8889       110
происшествие
     0.9000    0.9101    0.9050        89
     реклама
     0.8333    0.4000    0.5405        25
       семья
     0.7273    0.7273    0.7273        22
       спорт
     0.9333    0.9589    0.9459        73
      страна
     0.5789    0.4583    0.5116        24
     техника
     0.7361    0.8689    0.7970        61
   экономика
     0.8909    0.8448    0.8673        58

     accuracy                         0.8309       680
    macro avg     0.